In [32]:
import os
from pathlib import Path
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from scipy.stats import logistic
from scipy.stats import norm
from stargazer.stargazer import Stargazer
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import log_loss

import plotly
import plotly.express as px
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.templates.default = 'plotly'

warnings.filterwarnings("ignore")

In [33]:
path = Path(os.getcwd())
tech_prep = os.path.join(str(path), "utils/")
sys.path.append(tech_prep)
from py_helper_functions import *

In [34]:
# Loading data and checking
main_df = pd.read_csv('https://osf.io/4ay9x/download')
main_df.head().T

,0,1,2,3,4
Unnamed: 0,3,5,6,10,11
hhid,2600310997690,75680310997590,75680310997590,179140131100930,179140131100930
intmonth,January,January,January,January,January
stfips,AL,AL,AL,AL,AL
weight,3151.6801,3457.1138,3936.911,3288.364,3422.85
earnwke,1692.0,450.0,1090.0,769.23,826.92
uhours,40,40,60,40,40
grade92,43,41,41,40,43
race,1,2,2,1,1
ethnic,NaN,NaN,NaN,NaN,NaN


In [35]:
df = main_df.loc[main_df['occ2012'] == 2310] # Filtering for Elementary and middle school teachers

In [36]:
df.describe()

,Unnamed: 0,hhid,weight,earnwke,uhours,grade92,race,ethnic,age,sex,marital,ownchild,chldpres,occ2012
count,3636.000000,3.636000e+03,3636.000000,3636.000000,3636.000000,3636.000000,3636.000000,261.000000,3636.000000,3636.000000,3636.000000,3636.000000,3636.000000,3636.0
mean,159809.939769,4.480944e+14,2317.608004,1034.200594,40.691144,43.308306,1.277778,2.567050,42.160066,1.820132,2.536304,0.883938,2.360286,2310.0
std,92380.409835,3.202843e+14,1253.577145,528.825998,8.737564,1.129496,1.240695,2.385918,11.163914,0.384130,2.443091,1.104078,3.301203,0.0
min,43.000000,8.171510e+09,65.994300,0.230000,1.000000,34.000000,1.000000,1.000000,16.000000,1.000000,1.000000,0.000000,0.000000,2310.0
25%,79668.250000,1.408828e+14,1217.922700,692.300000,40.000000,43.000000,1.000000,1.000000,33.000000,2.000000,1.000000,0.000000,0.000000,2310.0
50%,161146.500000,4.103550e+14,2655.192200,961.000000,40.000000,43.000000,1.000000,1.000000,42.000000,2.000000,1.000000,0.000000,0.000000,2310.0
75%,241507.500000,7.219705e+14,3279.228425,1250.000000,40.000000,44.000000,1.000000,3.000000,51.000000,2.000000,5.000000,2.000000,4.000000,2310.0
max,317003.000000,9.998102e+14,10672.155900,2884.610000,80.000000,46.000000,21.000000,8.000000,64.000000,2.000000,7.000000,9.000000,15.000000,2310.0


In [37]:
df = df.loc[(main_df['uhours'] >= 20) # Filtering for at least 20 hours/week worked 
                 & (main_df['age'] >= 18) # Filtering for at least 18 years of age
                 & (main_df['earnwke'] > 0) # Filtering for more than 0 wage
                ]                     
df.shape

(3537, 23)

In [38]:
# Creating our target variable earnings per hour 'eph'

df['eph'] = df['earnwke'] / df['uhours']
df['eph'].describe()

count    3537.000000
mean       25.572499
std        12.586661
min         0.004107
25%        17.094000
50%        23.076750
75%        31.250000
max       100.125000
Name: eph, dtype: float64

## Exploring variables and creating dummies 
    1. Education level
    2. Age
    3. Gender
    4. Married
    5. has children
    6. Union
    7. Private or Public
    8. Race

In [39]:
# Exploring data

df.describe().round(2)

,Unnamed: 0,hhid,weight,earnwke,uhours,grade92,race,ethnic,age,sex,marital,ownchild,chldpres,occ2012,eph
count,3537.00,3.537000e+03,3537.00,3537.00,3537.00,3537.00,3537.00,256.00,3537.00,3537.00,3537.00,3537.00,3537.00,3537.0,3537.00
mean,159844.92,4.468632e+14,2317.60,1054.05,41.48,43.33,1.27,2.50,42.08,1.82,2.55,0.87,2.34,2310.0,25.57
std,92145.98,3.199704e+14,1256.61,518.83,7.41,1.10,1.21,2.34,11.12,0.39,2.45,1.09,3.29,0.0,12.59
min,43.00,8.171510e+09,65.99,0.23,20.00,34.00,1.00,1.00,18.00,1.00,1.00,0.00,0.00,2310.0,0.00
25%,79937.00,1.407641e+14,1217.92,711.53,40.00,43.00,1.00,1.00,33.00,2.00,1.00,0.00,0.00,2310.0,17.09
50%,161147.00,4.100094e+14,2649.98,961.53,40.00,43.00,1.00,1.00,42.00,2.00,1.00,0.00,0.00,2310.0,23.08
75%,241005.00,7.207065e+14,3283.37,1269.23,40.00,44.00,1.00,3.00,51.00,2.00,5.00,2.00,4.00,2310.0,31.25
max,317003.00,9.998102e+14,10672.16,2884.61,80.00,46.00,21.00,8.00,64.00,2.00,7.00,9.00,15.00,2310.0,100.12


In [54]:
# Checking what our school teachers education distribution

freq = df.groupby("grade92").agg(frequency=("grade92", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
grade92,,,
34,1,0.028,0.028
36,1,0.028,0.056
37,3,0.085,0.141
38,6,0.170,0.311
39,66,1.866,2.177
40,90,2.545,4.722
41,29,0.820,5.542
42,65,1.838,7.380
43,1518,42.918,50.298


In [57]:
# Checking what our school teachers gender distribution

freq = df.groupby("sex").agg(frequency=("sex", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
sex,,,
1,647,18.292,18.292
2,2890,81.708,100.000


In [58]:
# Checking what our school teachers marital status distribution

freq = df.groupby("marital").agg(frequency=("marital", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
marital,,,
1,2465,69.692,69.692
2,23,0.650,70.342
3,25,0.707,71.049
4,33,0.933,71.982
5,297,8.397,80.379
6,42,1.187,81.566
7,652,18.434,100.000


In [59]:
# Checking what our school teachers dependants(children) distribution

freq = df.groupby("ownchild").agg(frequency=("ownchild", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
ownchild,,,
0,1854,52.417,52.417
1,667,18.858,71.275
2,706,19.960,91.235
3,245,6.927,98.162
4,55,1.555,99.717
5,6,0.170,99.887
6,1,0.028,99.915
7,2,0.057,99.972
9,1,0.028,100.000


In [60]:
# Checking what our school teachers union membership distribution

freq = df.groupby("unionmme").agg(frequency=("unionmme", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
unionmme,,,
No,1697,47.979,47.979
Yes,1840,52.021,100.000


In [61]:
# Checking what our school teachers sector distribution

freq = df.groupby("class").agg(frequency=("class", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
class,,,
Government - Federal,22,0.622,0.622
Government - Local,2122,59.994,60.616
Government - State,689,19.480,80.096
"Private, For Profit",428,12.101,92.197
"Private, Nonprofit",276,7.803,100.000


In [ ]:
# Checking what our school teachers sector distribution

freq = df.groupby("class").agg(frequency=("class", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

In [41]:
# Creating Education variables

df['HS_GED'] = (df['grade92'] == 39).astype(int) # Associate degree (Vocational/occupational)
df['college_dropout'] = (df['grade92'] == 40).astype(int) # Associate degree (Vocational/occupational)
df['AD_V'] = (df['grade92'] == 41).astype(int) # Associate degree (Vocational/occupational)
df['AD_AP'] = (df['grade92'] == 42).astype(int) # Associate degree (Academic Program)
df['BD'] = (df['grade92'] == 43).astype(int) # Bachelor's degree (e.g.BA,AB,BS)
df['MD'] = (df['grade92'] == 44).astype(int) # Master's degree (e.g.MA,MS,MEng,Med,MSW,MBA) 
df['PD'] = (df['grade92'] == 45).astype(int) # Professional degree (e.g.MD,DDS,DVM,LLB,JD)
df['PhD'] = (df['grade92'] == 46).astype(int) # Doctorate degree(e.g.PhD,EdD)

In [42]:
# Creating a age squared variable

df["agesq"] = df["age"] ** 2 

In [43]:
# Creating female variable

df['female'] = np.where(df['sex'] == 2, '1', '0') 

In [44]:
# Creating dummy variables for marital status
# Created 3 variables by grouping each categorical value that made sense

df['married'] = np.where(df['marital'] <=2, '1', '0')
df['separated'] = np.where((df['marital'] >= 3) & (df['marital'] <= 6), '1', '0')
df['never_married'] = np.where(df['marital'] == 7, '1', '0')

In [45]:
# Creating dummy variable for having children
# Created 1 variable for having kids (1) or not (0) 

df['has_children'] = np.where(df['ownchild'] > 0, '1', '0')

In [46]:
# Creating dummy variable for union membership
# Created 1 variable for Yes (1) or No (0)

df['is_union_member'] = np.where(df['unionmme'] == 'Yes', '1', '0')

In [47]:
# Creating dummy variables for class/sector of employment
# Created 2 variables for government or private

df['government_employee'] = np.where((df['class'] == 'Government - Federal')
                                    |(df['class'] == 'Government - Local')
                                    |(df['class'] == 'Government - State'), '1', '0')

df['private_employee'] = np.where((df['class'] == 'Private, For Profit')
                                    |(df['class'] == 'Private, Nonprofit'), '1', '0')

In [48]:
# Creating dummy variables for race
# Created 2 variables for white person (1) or not (0)

df['white_person'] = np.where(df['race'] == 1, '1', '0')
df['non_white_person'] = np.where(df['race'] > 1 , '1', '0')

In [49]:
# Creating dummy variables for citizenship
# Created 2 variables for native or foreign

df['us_native'] = np.where((df['prcitshp'] == 'Native, Born In US')
                                    |(df['prcitshp'] == 'Native, Born Abroad Of US Parent(s)')
                                    |(df['prcitshp'] == 'Native, Born in PR or US Outlying Area'), '1', '0')

df['us_foreign'] = np.where((df['prcitshp'] == 'Foreign Born, US Cit By Naturalization')
                                    |(df['prcitshp'] == 'Foreign Born, Not a US Citizen'), '1', '0')

In [50]:
df.head().T

,21,50,84,130,145
Unnamed: 0,43,116,199,286,324
hhid,954001919079770,310343067909068,678011962039080,24370720695699,299960537490630
intmonth,January,January,January,January,January
stfips,AL,AL,AL,AK,AK
weight,2847.5801,2690.8661,2424.1925,341.9676,414.7041
earnwke,826.92,673.07,1000.0,520.0,769.23
uhours,40,45,56,40,50
grade92,44,43,44,39,43
race,1,1,2,1,1
ethnic,NaN,NaN,NaN,NaN,NaN
